In [ ]:
import pandas as pd

df=pd.read_csv('./dataset/text/text.csv')
df

In [ ]:
img_text_pairs = dict(zip(df['image_name'], df['text_explanation']))
img_text_pairs

In [ ]:
import os
mllm_data=[]
base_img_path='/home/chuannong1/KJY/MultiLLM/dataset/image/'
for i in img_text_pairs:
    dict_temp=dict()
    dict_temp['messages']=[{"content": "<image>根据上传的医学影像进行诊断","role": "user"},
                           {"content": img_text_pairs[i],"role": "assistant"}]
    dict_temp['images']=[os.path.join(base_img_path,str(i)+'.jpg')]
    mllm_data.append(dict_temp)

mllm_data

# 预训练

In [5]:
import os

def prompt_template_pt(img_path,text):
    dict_temp=dict()
    dict_temp['messages']=[{"content": "<image>医学影像诊断结果:","role": "user"},
                           {"content": text,"role": "assistant"}]
    dict_temp['images']=[img_path]

    return dict_temp

In [6]:
import pandas as pd

def img_text_pair_pt(data_dir):
    mllm_data_temp=[]
    patient_img_dir=os.path.join(data_dir,'datajpg')
    patient_id_list=os.listdir(patient_img_dir)
    df_patient_fp=os.path.join(data_dir,'病理描述文本.xlsx')
    df_patient=pd.read_excel(df_patient_fp)
    for patient_id in patient_id_list:
        patient_id_img_dir=os.path.join(patient_img_dir,patient_id)
        for img_id in os.listdir(patient_id_img_dir):
            img_path=os.path.join(patient_id_img_dir,img_id)
            patient_id_int = int(patient_id)

            if patient_id_int in df_patient['id'].values:
                text = df_patient[df_patient['id'] == patient_id_int]['text'].values[0]
                dict_temp=prompt_template_pt(img_path,text)
                mllm_data_temp.append(dict_temp)

    return mllm_data_temp

In [7]:
base_path="/mnt/newdisk/MultiLLMdata/data/train"
mllm_data=[]
for dir_name in os.listdir(base_path):
    print(f'{dir_name}开始处理')
    dir_path=os.path.join(base_path,dir_name)
    mllm_data = mllm_data + img_text_pair_pt(dir_path)
    print(f'{dir_name}完成')

chongyisan开始处理
chongyisan完成
jidafuyuan4开始处理
jidafuyuan4完成
luzhouzhongyi6开始处理
luzhouzhongyi6完成
chongyier1开始处理
chongyier1完成
nanchongzhongxinyiyuan8开始处理
nanchongzhongxinyiyuan8完成
leshanzhongyi5开始处理
leshanzhongyi5完成
fudanzhongshan3开始处理
fudanzhongshan3完成
mianyangsanyuan7开始处理
mianyangsanyuan7完成
chongyiyi2开始处理
chongyiyi2完成


In [8]:
import json

# 将字典转换为 JSON 字符串
json_string = json.dumps(mllm_data, ensure_ascii=False, indent=4)
# 如果需要将 JSON 保存到文件
with open('./LLaMA-Factory/data/mllm_pt.json', 'w', encoding='utf-8') as f:
    json.dump(mllm_data, f, ensure_ascii=False, indent=4)

# LoRA微调

In [1]:
import os

def prompt_template(img_path,text):
    dict_temp=dict()
    dict_temp['messages']=[{"content": "You are a medical imaging diagnosis assistant. Please answer user questions in professional language.","role": "system"},
                           {"content": "<image>Diagnose based on uploaded medical images:","role": "user"},
                           {"content": text,"role": "assistant"}]
    dict_temp['images']=[img_path]

    return dict_temp

In [2]:
import pandas as pd

def img_text_pair(data_dir):
    mllm_data_temp=[]
    patient_img_dir=os.path.join(data_dir,'datajpg')
    patient_id_list=os.listdir(patient_img_dir)
    df_patient_fp=os.path.join(data_dir,'病理描述文本en.xlsx')
    df_patient=pd.read_excel(df_patient_fp)
    for patient_id in patient_id_list:
        patient_id_img_dir=os.path.join(patient_img_dir,patient_id)
        for img_id in os.listdir(patient_id_img_dir):
            img_path=os.path.join(patient_id_img_dir,img_id)
            patient_id_int = int(patient_id)

            if patient_id_int in df_patient['id'].values:
                text = df_patient[df_patient['id'] == patient_id_int]['text'].values[0]
                dict_temp=prompt_template(img_path,text)
                mllm_data_temp.append(dict_temp)

    return mllm_data_temp

In [3]:
base_path="/mnt/newdisk/MultiLLMdata/data/val"
mllm_data=[]
for dir_name in os.listdir(base_path):
    print(f'{dir_name}开始处理')
    dir_path=os.path.join(base_path,dir_name)
    mllm_data = mllm_data + img_text_pair(dir_path)
    print(f'{dir_name}完成')

yibineryuan9开始处理
yibineryuan9完成


In [4]:
import json

# 将字典转换为 JSON 字符串
json_string = json.dumps(mllm_data, ensure_ascii=False, indent=4)
# 如果需要将 JSON 保存到文件
with open('./LLaMA-Factory/data/mllm_val_en.json', 'w', encoding='utf-8') as f:
    json.dump(mllm_data, f, ensure_ascii=False, indent=4)

## MS-SWIFT

In [7]:
import os

def prompt_template(img_path_list,text,seq_len):
    dict_temp=dict()
    img_special_token='<image>'
    dict_temp={"system": "你是医学影像诊断小助手,请用专业的语言回答用户问题",
               "query": f"这是一串连续的医学影像：{img_special_token*seq_len}。请根据上传医学影像进行诊断",
               "response": text,
               "history": [],
               "images":img_path_list}

    return dict_temp

In [2]:
import pandas as pd
import re

def img_text_pair(data_dir):
    mllm_data_temp=[]
    patient_img_dir=os.path.join(data_dir,'datajpg')
    patient_id_list=os.listdir(patient_img_dir)
    df_patient_fp=os.path.join(data_dir,'病理描述文本.xlsx')
    df_patient=pd.read_excel(df_patient_fp)
    for patient_id in patient_id_list:
        img_path_list=[]
        patient_id_img_dir=os.path.join(patient_img_dir,patient_id)
        for img_id in sorted(os.listdir(patient_id_img_dir),key=lambda x: int(re.findall(r'\d+', x)[0])):
            img_path=os.path.join(patient_id_img_dir,img_id)
            patient_id_int = int(patient_id)
            img_path_list.append(img_path)

        if patient_id_int in df_patient['id'].values:
            text = df_patient[df_patient['id'] == patient_id_int]['text'].values[0]
        
        seq_len=len(img_path_list)
        dict_temp=prompt_template(img_path_list,text,seq_len)
        mllm_data_temp.append(dict_temp)

    return mllm_data_temp

In [3]:
mllm_data_temp=img_text_pair("/mnt/newdisk/MultiLLMdata/data/train/chongyier1/")

In [4]:
base_path="/mnt/newdisk/MultiLLMdata/data/train"
mllm_data=[]
for dir_name in os.listdir(base_path):
    print(f'{dir_name}开始处理')
    dir_path=os.path.join(base_path,dir_name)
    mllm_data = mllm_data + img_text_pair(dir_path)
    print(f'{dir_name}完成')

chongyisan开始处理
chongyisan完成
jidafuyuan4开始处理
jidafuyuan4完成
luzhouzhongyi6开始处理
luzhouzhongyi6完成
chongyier1开始处理
chongyier1完成
nanchongzhongxinyiyuan8开始处理
nanchongzhongxinyiyuan8完成
leshanzhongyi5开始处理
leshanzhongyi5完成
fudanzhongshan3开始处理
fudanzhongshan3完成
mianyangsanyuan7开始处理
mianyangsanyuan7完成
chongyiyi2开始处理
chongyiyi2完成


In [6]:
import json

# 将列表转换为 JSON 并写入文件
with open("./ms-swift/data/qwen2_vl_train.json", "w", encoding="utf-8") as json_file:
    for item in mllm_data:
        json.dump(item, json_file, ensure_ascii=False)
        json_file.write("\n")  # 每个对象占一行

In [7]:
base_path="/mnt/newdisk/MultiLLMdata/data/val"
mllm_data=[]
for dir_name in os.listdir(base_path):
    print(f'{dir_name}开始处理')
    dir_path=os.path.join(base_path,dir_name)
    mllm_data = mllm_data + img_text_pair(dir_path)
    print(f'{dir_name}完成')

yibineryuan9开始处理
yibineryuan9完成


In [8]:
import json

# 将列表转换为 JSON 并写入文件
with open("./ms-swift/data/qwen2_vl_val.json", "w", encoding="utf-8") as json_file:
    for item in mllm_data:
        json.dump(item, json_file, ensure_ascii=False)
        json_file.write("\n")  # 每个对象占一行